# **Housing in California**

In this tuotiral, we used a guide from the machine learning course taught by Guowei Wei at MSU. This dataset came with Google Colab, so we tried to do some anaylsis on it.  We'll try Linear Regression and implement different algorithms later. 



# About the Dataset

This dataset is from 1990 California Census data, which originally appeared in it's current form in a 1997 paper titled "Sparse Spatial Autoregressions" by R. Kelley Pace and Ronald Barry. The variables in this dataset are longitude, latitutde, median housing price, total rooms, total bedrooms, population per block, household income (in units of 10,000 USD), ocean proximity, and median house value.

#Importing Libraries

In [0]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.preprocessing import StandardScaler

#Putting Our Data Into a Pandas Dataframe

In [0]:
train_data = pd.read_csv('/content/sample_data/california_housing_train.csv')

X_train_dataframe = train_data.drop(columns = ['longitude', 'latitude' ,'median_house_value'])
X_train = X_train_dataframe.values

y_train_dataframe = train_data['median_house_value']
y_train = y_train_dataframe.values

test_data = pd.read_csv('/content/sample_data/california_housing_test.csv')

X_test_dataframe = test_data.drop(columns = ['longitude', 'latitude' ,'median_house_value'])
X_test = X_test_dataframe.values

y_test_dataframe = test_data['median_house_value']
y_test = y_test_dataframe.values

In [0]:
scaler = StandardScaler() # call an object function
scaler.fit(X_train)   # calculate mean
X_train_norm = scaler.transform(X_train)  # apply normalization on X_train
X_test_norm = scaler.transform(X_test)    # apply normalization on X_test

In [0]:
print(X_train_norm.shape)
print(X_test_norm.shape)
print(y_train.shape)
print(y_test.shape)

(17000, 6)
(3000, 6)
(17000,)
(3000,)


# The Theory of Regression With Regularization

We'll start by implementing linear regression with a regularization. In particular, we'll implement a ridge regression since LASSO will take a very similar approach.

We also choose a method involving regularization since we can't solve for the optimal weights using analytical methods. Instead, we'll have to implement numerical methods using gradient descent, which fits more with modern methods used for deep learning.

Since this is a linear regression, we assume that our data can be reasonably modeled by a straight line in some multidimensional space, let's say
$$\hat{y} = XW + b.$$
In this case, $\hat{y}$ is our predicted output, which is the median housing price in a block. The vector $b$ represents the bias. This tell us the difference between the true values and the predicted values in our model. This will be a vector where the value of each entry is the same since it affects our whole dataset globally.

Assume our training set has $N$ datapoints and we have $D$ features. Then let $X$ be the matrix which contains each feature as the column entries and each datapoints is the row. The vector $W$ is the weight associated with each feature. Thus, $X$ has shape $(N,D)$ and  $W$ has shape $(D,1)$, 

We want the set of weights and bias, $W$ and $b$, that allows us to make the best prediction on our test data. However, how can we measure what is the best fit? We'll check the squared different between our predicted housing prices in a block and our actual housing prices in a block, which we'll call $y$. That is, our loss function will be:
$$L = \frac{\|\hat{y} - y\|_2^2}{2N} = \frac{1}{2N} \sum_{i = 1}^{N} (\hat{y}_i - y_i)^2.$$
To account for overfitting, we'll introduce a regularization parameter, $\alpha$ and let our new loss function be
$$L = \boxed{\frac{\|\hat{y} - y\|_2^2}{2N} + \frac{\alpha}{2N} ||W||_2^2} = \frac{1}{2N} \sum_{i = 1}^N (\hat{y}_i - y_i)^2 + \frac{\alpha}{2N} \sum_{n=1}^{N} ||W_n||_2^2$$
For simplicity, we'll code this in terms of the vector versions rather than working element by element.

We're implementing this with gradient descent. Our unknown parameters we want to find are the matrix $W$ and the value $b$, so we'll find the partial derivatives of our loss function with respect to the variables for use in the back propagation. First, let's start with $W$:
$$\dfrac{\partial{L}}{\partial{W}} = \dfrac{\partial{L}}{\partial{\hat{y}}}\dfrac{\partial{\hat{y}}}{\partial{W}} = \frac{1}{N} X^T (\hat{y}-y) + \frac{\alpha}{N}W.$$
Next, let's do this with $b$:
$$\dfrac{\partial{L}}{\partial{W}} = \dfrac{\partial{L}}{\partial{\hat{y}}}\dfrac{\partial{\hat{y}}}{\partial{W}} = \frac{1}{N} X^T (\hat{y}-y) + \frac{\alpha}{N}W.$$
Since we're implementing gradient descent, we need a rate at which our algorithm "learns," which we'll call $\beta$. In more mathematical terms, this is just a scaling factor on our parial derivative that we tweak to find our minimum. We'll update our weights and bias in the following way:
  $$W_{new} = W_{old} - \beta\dfrac{\partial{L}}{\partial{W}} = \boxed{W_{old} - \frac{\beta}{N} X^T (\hat{y}-y) - \frac{\beta\alpha}{N}W,}$$
  $$b_{new} = b_{old} - \beta\dfrac{\partial{L}}{\partial{b}} = \boxed{b_{old} - \frac{\beta}{N} \sum_{n=1}^{N}(\hat{y}_n - y_n).}$$
This is starting to look more like something we can code up now!

# Forward Propagation

This is the easy part! We'll start by calculating what our prediction would be:

In [0]:
def forward(X, W, b):
    y_hat = np.dot(X, W) + b #adds the scalar b to each entry of our vector
    return y_hat

# Backward Propagation

So, our initial guess probably isn't the minimzing guess, so what we'll do is update our guess for $W$ and $b$ by using gradient descent:

In [0]:
def vec_sub(y, y_hat):
    z = np.zeros((len(y), 1), dtype = 'float')
    for n in range(len(y)):
        z[n] = y_hat[n] - y[n]
    return z

def gradient_descent(W, b, y, y_hat, X, alpha, beta):
    z = vec_sub(y,y_hat)/X.shape[0]
    grad_W = np.dot(X.T, z)  + alpha / X.shape[0] * W
    grad_b = np.sum(z, axis = 0, keepdims = True)
    W = W - beta * grad_W
    b = b - beta * grad_b
    return W, b

# Prediction

Now that we have all the pieces for our algorithm, what's left is to predict after running through enough iterations. We'll take our test data and assume it fits the model. Then we have
$$y_{predicted} = X_{test}W_{optimal} + b_{optimal}.$$

In [0]:
def predict(W, X_test, b):
    y_pred = np.dot(X_test, W) + b #adds the scalar b to each entry of our vector
    return y_pred

# Checking the Loss

We need to check if our code is actually minimizing our loss, so let's write a function print out the loss:


In [0]:
def print_loss(y_hat, y):
    return np.linalg.norm(vec_sub(y,y_hat))**2/len(y)

# Putting it Together

Okay, so now that we have all the required pieces, let's put it all together. As with any numerical algorithm, let's put this through multiple iterations in a loop and then predict:

In [0]:
alpha = 1
beta = 0.01
W = np.zeros((X_train_norm.shape[1],1), dtype = 'float')
print(W.shape)
y_hat = np.zeros((X_train_norm.shape[0],1), dtype = 'float')
b = 0.0
iterations = 10000 #You'll probably need more iterations to get a good result, but google doesn't really give much run time.
for n in range(iterations):
    y_hat = forward(X_train_norm, W, b)
    W, b = gradient_descent(W, b, y_train, y_hat, X_train_norm, alpha, beta)
    loss = print_loss(y_hat,y_train)
    print(loss)
pred = predict(W, X_test_norm, b)
pred = np.matrix.round(pred)

Streaming output truncated to the last 5000 lines.
358666442031548.7
358666854707531.6
358667267113625.3
358667679250097.44
358668091117098.4
358668502714846.1
358668914043339.4
358669325102895.1
358669735893641.1
358670146415871.7
358670556669615.94
358670966655086.8
358671376372452.56
358671785821895.7
358672195003602.44
358672603917726.75
358673012564504.44
358673420944016.94
358673829056460.7
358674236902011.1
358674644480884.1
358675051793225.4
358675458839253.44
358675865618972.7
358676272132775.25
358676678380643.8
358677084362911.9
358677490079554.9
358677895531036.9
358678300717241.8
358678705638570.6
358679110295015.94
358679514686800.8
358679918814092.94
358680322677181.56
358680726276035.8
358681129611038.1
358681532682156.44
358681935489655.2
358682338033767.25
358682740314581.25
358683142332330.8
358683544087102.3
358683945579096.8
358684346808544.06
358684747775647.5
358685148480249.3
358685548922895.0
358685949103705.06
358686349022627.25
358686748680055.44
358687148076

# How accurate is This?

How accurate is this now? Let's check:

In [0]:
print(y_test)
print(pred)


[344700. 176500. 270500. ...  62000. 162500. 500001.]
[[331639.]
 [215505.]
 [276310.]
 ...
 [ 87110.]
 [186718.]
 [435743.]]


# Conclusion and Discussions

Not bad for linear regression. We could probably feature engineer this to make it a little better, but we probably won't get anything that accurate. We've erased any variables about location since it definitely isn't a variable that scales linearly.  However, location is definitely an important part of housing price. This problem probably won't be solved well using regression, so let's try something a little different in our next notebook.